In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt



In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64

# create data loader 
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)



In [ ]:
for x, y in test_dataloader:
    print("shape of x: [batch, channel, height, width]:", x.shape )
    print("shaope of y:", y.shape, y.type)
    break 

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, ):
        super (NeuralNetwork, self, ).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(), 
            nn.Linear(512, 512), 
            nn.ReLU(), 
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
        

In [7]:
model = NeuralNetwork().to(device)


In [8]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [16]:
def train (data_loader, model, loss_fn, optimizer):
    size = len(data_loader.dataset)
    model.train()
    for batch, (X, y) in enumerate (data_loader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

In [21]:
def test(data_loader, model, loss_fn, ):
    size = len(data_loader.dataset)
    num_batch = len(data_loader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batch
        correct /= size 
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.919186  [    0/60000]
loss: 1.884700  [ 6400/60000]
loss: 1.771096  [12800/60000]
loss: 1.798242  [19200/60000]
loss: 1.701908  [25600/60000]
loss: 1.653175  [32000/60000]
loss: 1.665179  [38400/60000]
loss: 1.580075  [44800/60000]
loss: 1.602161  [51200/60000]
loss: 1.476555  [57600/60000]
Test Error: 
 Accuracy: 60.8%, Avg loss: 1.513432 

Epoch 2
-------------------------------
loss: 1.587751  [    0/60000]
loss: 1.548025  [ 6400/60000]
loss: 1.403701  [12800/60000]
loss: 1.463510  [19200/60000]
loss: 1.357545  [25600/60000]
loss: 1.349653  [32000/60000]
loss: 1.357232  [38400/60000]
loss: 1.296346  [44800/60000]
loss: 1.323775  [51200/60000]
loss: 1.212294  [57600/60000]
Test Error: 
 Accuracy: 63.6%, Avg loss: 1.249787 

Epoch 3
-------------------------------
loss: 1.328118  [    0/60000]
loss: 1.308408  [ 6400/60000]
loss: 1.145133  [12800/60000]
loss: 1.247423  [19200/60000]
loss: 1.126288  [25600/60000]
loss: 1.148171  [32000/600